In [1]:
import os
import pandas as pd
import jupyter_black

jupyter_black.load()

current_dir = os.path.dirname(os.path.abspath("__file__"))

### Set file path to datasets

In [16]:
pokemon_path = os.path.join(current_dir, "data", "pokemon.csv")
google_path = os.path.join(current_dir, "data", "google_stock_price.csv")
nba_path = os.path.join(current_dir, "data", "nba.csv")
revenue_path = os.path.join(current_dir, "data", "revenue.csv")

#### Go back and forth between `pd.Series` and `pd.DataFrame` with `to_frame()` and `squeeze()`

In [6]:
fruits = ["Apple", "Orange", "Banana", "Pear"]
weekdays = ["Monday", "Tuesday", "Wednesday", "Thursday"]
s = pd.Series(index=weekdays, data=fruits)
display(s)
df = s.to_frame()
display(df)
s = df.squeeze()
display(s)

Monday        Apple
Tuesday      Orange
Wednesday    Banana
Thursday       Pear
dtype: object

,0
Monday,Apple
Tuesday,Orange
Wednesday,Banana
Thursday,Pear


Monday        Apple
Tuesday      Orange
Wednesday    Banana
Thursday       Pear
Name: 0, dtype: object

### The `apply()` Method

In [15]:
pokemon = pd.read_csv(pokemon_path, index_col="Pokemon").squeeze()


def rank_pokemon(pokemon_type):
    if pokemon_type in ["Grass", "Fire", "Water"]:
        return "Classic"
    elif pokemon_type == "Normal":
        return "Boring"
    return "TBD"


pokemon.apply(rank_pokemon).head(3)

Pokemon
Bulbasaur    Classic
Ivysaur      Classic
Venusaur     Classic
Name: Type, dtype: object

### Insert new column at a specific position

In [24]:
nba = pd.read_csv(nba_path)
nba.insert(3, "new_col", "New column at position 3")
nba.head(3)

,Name,Team,Number,new_col,Position,Age,Height,Weight,College,Salary
0,Avery Bradley,Boston Celtics,0.0,New column at position 3,PG,25.0,6-2,180.0,Texas,7730337.0
1,Jae Crowder,Boston Celtics,99.0,New column at position 3,SF,25.0,6-6,235.0,Marquette,6796117.0
2,John Holland,Boston Celtics,30.0,New column at position 3,SG,27.0,6-5,205.0,Boston University,NaN
